In [1]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn import datasets
from sklearn.datasets import fetch_mldata
import time
%matplotlib inline  
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
import skimage.io as io
import skimage.transform as transform
from sklearn import metrics
from skimage import filters
from skimage.transform import SimilarityTransform
import random

#import 

In [2]:
def euclidean_distance(X, Y):
    return np.sqrt(np.abs((X ** 2).sum(axis=1).reshape(-1, 1) +
                   (Y ** 2).sum(axis=1) - 2 * np.dot(X, Y.T)))

def cosine_distance(X, Y):
    return 1 - np.dot(X, Y.T) / np.sqrt((X
            ** 2).sum(axis=1)).reshape(-1, 1) / np.sqrt((Y
            ** 2).sum(axis=1))

In [3]:
class KNNClassifier:
    def __init__(self, k, strategy, metric,
                 weights, test_block_size):
        self.k = k
        self.strategy = strategy
        self.metric = metric
        self.weights = weights
        self.test_block_size = test_block_size
        if self.strategy != 'my_own':
            self.knn_classifier = NearestNeighbors(n_neighbors = self.k, 
                                                   algorithm=self.strategy, 
                                                   metric = self.metric)
        self.eps = 1e-5
    
    def fit(self, X, y):
        self.y = np.copy(y).astype(int)
        if self.strategy != 'my_own':
            self.knn_classifier.fit(X, y)
        else:
            self.X = np.copy(X).astype(float)

            
    def find_kneighbors(self, X, return_distance):
        test = np.copy(X)
        if self.strategy != 'my_own':
            dist, idx = self.knn_classifier.kneighbors(test, n_neighbors = self.k)
            dist = dist.reshape(-1, self.k)
            idx = idx.reshape(-1, self.k)
        else:
            if self.metric == 'euclidean':
                d = euclidean_distance(test, self.X)
            else:
                d = cosine_distance(test, self.X)
            print(8)
            idx = np.argsort(dist, axis = 1)[:, :self.k]
            dist = a = [a[i][j] for i, j in enumerate(idx)]
            print(10)
        if return_distance:
            return (dist, idx.astype(int))
        else:
            return idx.astype(int)
    
    def predict(self, X):
        self.eps = 1e-5
        pred = np.zeros(0).astype(int)
        for i in range(0, X.shape[0], self.test_block_size):
            test = np.copy(X[i : i + self.test_block_size])
            
            if not self.weights:
                idx = self.find_kneighbors(test, return_distance = False)
                classes = self.y[idx]
                p = np.array([np.bincount(classes[i]).argmax() for i in range(classes.shape[0])]).astype(int)
                pred = np.hstack((pred, p))
            else:
                dist, idx = self.find_kneighbors(test, return_distance = True)
                w = 1 / (dist + self.eps)
                classes = self.y[idx]
                p = np.array([np.bincount(classes[i], w[i]).argmax() for i in range(classes.shape[0])]).astype(int)
                pred = np.hstack((pred, p))
            
        return pred     
    
    
    


In [4]:
def kfold(n, n_folds):
    folds = []
    size = n // n_folds
    pos_valid = 0 
    for i in range(n_folds):
        if (i >= n % n_folds):
            valid = np.arange(pos_valid, pos_valid + size)
            train = np.arange(0, pos_valid)
            train = np.hstack((train, np.arange(pos_valid + size, n)))
            pos_valid = pos_valid + size
        else:
            valid = np.arange(pos_valid, pos_valid + size + 1)
            train = np.arange(0, pos_valid)
            train = np.hstack((train, np.arange(pos_valid + size + 1, n)))
            pos_valid = pos_valid+ size + 1
        folds.append((train, valid))
    return folds

def knn_cross_val_score(X, y, k_list, score, cv, **kwargs):
    if cv == None:
            cv = kfold(X.shape[0], 3)
    if score == 'accuracy':
        m = KNNClassifier(max(k_list), **kwargs)
        sc = {}
        
        for fold in cv:
            train = fold[0]
            valid = fold[1]
            x_val = X[valid]
            y_val = y[valid].astype(int)
            y_train = y[train].astype(int)

            m.fit(X[train], y_train)
            if not kwargs['weights']:
                idx = m.find_kneighbors(x_val, return_distance = False)
                w = np.ones((idx.shape[0], idx.shape[1]))
            else:
                dist, idx = m.find_kneighbors(x_val, return_distance = True)
                w = 1 / (dist + 1e-5)

            classes = y_train[idx]
            for k in k_list:
                p = np.array([np.bincount(classes[i][:k], w[i][:k]).argmax()
                             for i in range(classes.shape[0])]).astype(int)
                if k in sc:
                    sc[k] = np.hstack((sc[k], np.array(np.mean(y_val == p))))
                else:
                    sc[k] = np.array([np.mean(y_val == p)])
        return sc
    elif score == 'clock':
        m = KNNClassifier(max(k_list), **kwargs)
        t1 = time.perf_counter()
        for i, fold in enumerate(cv):
            train = fold[0]
            valid = fold[1]
            x_val = X[valid]
            y_train = y[train].astype(int)
            m.fit(X[train], y[train])
            p = m.predict(x_val)
        t2 = time.perf_counter()
        return t2 - t1

In [5]:
mnist = fetch_mldata('MNIST original')
X_train = mnist.data[:10000]
y_train = mnist.target[:10000]
X_test = mnist.data[60000:]
y_test = mnist.target[60000:]

Ошибки связаны с изменением наклона, слиянием частей цифры, недорисованными линиями

In [6]:
changes = {'rotate', 'shift_lr', 'shift_ud', 'gauss'}
rot = [0, 5, 10, 15]
shift = [0, 1, 2, 3]
gauss = [0, 0.5, 1, 1.5]


In [7]:
def change_im(X_train, name, v):
    X = np.copy(X_train)
    
    if name == 'gauss':
        res = np.empty((X.shape[0] * 2, X.shape[1]))
        res[0:X.shape[0]] = np.copy(X_train)
        pos = X.shape[0]
        for im in X:
            res[pos] = filters.gaussian(im.reshape(-1, 28), v).reshape(1, -1)
            pos += 1
            
    if name == 'rotate':
        res = np.empty((X.shape[0] * 3, X.shape[1]))
        res[0:X.shape[0]] = np.copy(X_train)
        pos = X.shape[0]
        for im in X:
            res[pos] = transform.rotate(im.reshape(-1, 28), v).reshape(1, -1)
            pos += 1     
        for im in X:
            res[pos] = transform.rotate(im.reshape(-1, 28), -v).reshape(1, -1)
            pos += 1  
            
    if name == 'shift':
        res = np.empty((X.shape[0] * 5, X.shape[1]))
        res[0:X.shape[0]] = np.copy(X_train)
        pos = X.shape[0]
        for im in X:
            tform = SimilarityTransform(translation=(0, v))
            res[pos] = transform.warp(im.reshape(-1, 28), tform).reshape(1, -1)
            pos += 1  
        for im in X:
            tform = SimilarityTransform(translation=(0, -v))
            res[pos] = transform.warp(im.reshape(-1, 28), tform).reshape(1, -1)
            pos += 1  
        for im in X:
            tform = SimilarityTransform(translation=(v, 0))
            res[pos] = transform.warp(im.reshape(-1, 28), tform).reshape(1, -1)
            pos += 1  
        for im in X:
            tform = SimilarityTransform(translation=(-v, 0))
            res[pos] = transform.warp(im.reshape(-1, 28), tform).reshape(1, -1)
            pos += 1  
           
    return res
            
        

In [8]:
def cross_val(X_t, y_t, k=4, **kwargs):
    
    iters = 100
    size = X_t.shape[0] * 2 // iters // 3 
    
    cv = kfold(X_t.shape[0], 3)
    m = KNNClassifier(k, **kwargs)
    
    sc = []
    pred = []

    for fold in cv:
        distance = np.empty((X_t.shape[0] // 3, k * iters))
        classes = np.empty((X_t.shape[0] // 3, k * iters))        
            
        train = fold[0]
        valid = fold[1]
        x_val = X_t[valid]
        y_val = y_t[valid].astype(int)
        y_train = y_t[train].astype(int)
        x_train = X_t[train]
        
    
        for i in range(iters):
            X = x_train[i * size: i * size + size, :]
            y = y_train[i * size: i * size + size]
            m.fit(X, y)
            dist, idx = m.find_kneighbors(x_val, return_distance=True)
            distance[:, i * k : i * k + k] = dist
            classes[:, i * k : i * k + k] = y[idx]
        
        d1 = np.copy(distance)
        dist = (np.sort(distance, axis=1))[:, :k]
        cl = (np.argsort(d1, axis = 1))[:, :k]
        classes = np.array([classes[i][j] for i, j  in enumerate(cl)]).astype(int)

        classes = classes[:, :k]
        w = 1/(dist + 1e-5)
            
            
        p = np.array([np.bincount(classes[i], w[i]).argmax()
                     for i in range(classes.shape[0])]).astype(int)

        sc.append(([np.mean(y_val == p)]))
        pred.append(p)
    return sc, pred

In [13]:
def cross_val_pred(X_t, y_t, x_val, y_val, k=4, **kwargs):
    
    iters = 100
    size = X_t.shape[0] // iters
    
    m = KNNClassifier(k, **kwargs)

    distance = np.empty((x_val.shape[0], k * iters))
    classes = np.empty((x_val.shape[0], k * iters))        

    y_train = y_t.astype(int)
    x_train = X_t

    for i in range(iters):
        X = x_train[i * size: i * size + size, :]
        y = y_train[i * size: i * size + size]
        m.fit(X, y)
        dist, idx = m.find_kneighbors(x_val, return_distance=True)
        distance[:, i * k : i * k + k] = dist
        classes[:, i * k : i * k + k] = y[idx]

    d1 = np.copy(distance)
    dist = (np.sort(distance, axis=1))[:, :k]
    cl = (np.argsort(d1, axis = 1))[:, :k]
    classes = np.array([classes[i][j] for i, j  in enumerate(cl)]).astype(int)

    classes = classes[:, :k]
    w = 1/(dist + 1e-5)

    p = np.array([np.bincount(classes[i], w[i]).argmax()
                 for i in range(classes.shape[0])]).astype(int)

    return np.mean(y_val == p), p

In [10]:
mnist = fetch_mldata('MNIST original')
X_train = mnist.data[:60000]
y_train = mnist.target[:60000]

X = np.hstack((X_train, y_train.reshape(-1, 1)))
np.random.shuffle(X)
X_train = X[:, :784]
y_train = X[:, 784:]

X_test = mnist.data[60000:]
y_test = mnist.target[60000:]
y_train = y_train.reshape(60000,)



In [13]:
score = []
pred = []

cv = kfold(60000, 3)

for g in gauss[1:]:
    sc = []
    for fold in cv:
        X_t = X_train[fold[0]]
        y_t = y_train[fold[0]]
        y = np.copy(y_t)
        for i in range(1):
            y = np.concatenate((y, y_t))
        X = change_im(X_t, 'gauss', g)
        print(1)
        res, p = cross_val_pred(X, y, X_train[fold[1]],y_train[fold[1]],  k = 4, strategy = 'brute', metric = 'cosine',  
                    weights = True, test_block_size = 1024)
        print(res)
        sc.append(res)
    score.append(np.mean(sc))
score

1
0.9761
1
0.97675
1
0.97595
1
0.97975
1
0.98055
1
0.97985
1
0.9778
1
0.97955
1
0.9785


[0.9762666666666666, 0.98005, 0.9786166666666666]

In [14]:
score = []
pred = []

cv = kfold(60000, 3)

for r in rot[1:]:
    sc = []
    for fold in cv:
        X_t = X_train[fold[0]]
        y_t = y_train[fold[0]]
        y = np.copy(y_t)
        for i in range(2):
            y = np.concatenate((y, y_t))
        X = change_im(X_t, 'rotate', r)
        print(1)
        res, p = cross_val_pred(X, y, X_train[fold[1]],y_train[fold[1]],  k = 4, strategy = 'brute', metric = 'cosine',  
                    weights = True, test_block_size = 1024)
        print(res)
        sc.append(res)
    score.append(np.mean(sc))
score
     

1
0.98025
1
0.9811
1
0.9812
1
0.9808
1
0.9819
1
0.98045
1
0.97815
1
0.98085
1
0.9779


[0.9808499999999999, 0.9810500000000001, 0.9789666666666667]

In [15]:
y_t = y_train[:51000]
X_t = X_train[:51000]
    
score = []
pred = []

cv = kfold(60000, 3)

for s in shift[1:]:
    sc = []
    for fold in cv:
        X_t = X_train[fold[0]]
        y_t = y_train[fold[0]]
        y = np.copy(y_t)
        for i in range(4):
            y = np.concatenate((y, y_t))
        X = change_im(X_t, 'shift', s)
        print(1)
        res, p = cross_val_pred(X, y, X_train[fold[1]],y_train[fold[1]],  k = 4, strategy = 'brute', metric = 'cosine',  
                    weights = True, test_block_size = 1024)
        print(res)
        sc.append(res)
    score.append(np.mean(sc))
score
     

1
0.9808
1
0.98265
1
0.98195
1
0.97695
1
0.97845
1
0.9788
1
0.9749
1
0.9768
1
0.97675


[0.9817999999999999, 0.9780666666666668, 0.97615]

In [24]:
pred_s = np.copy(pred)

In [38]:
X = change_im(X_train, 'shift', 1)
y = np.copy(y_train)
for i in range(4):
    y = np.concatenate((y, y_train))
print(1)

m = KNNClassifier(4, 'brute', 'cosine', True, 256)
m.fit(X, y)
print(2)
res = m.predict(X_test)

1
2


In [36]:
X.shape

(300000, 784)

In [39]:
np.mean(res == y_test)

0.9798

In [40]:
res_shift = np.copy(res)

In [42]:
mist_shift = metrics.confusion_matrix(y_test, res)
mist_shift

array([[ 977,    0,    0,    0,    0,    0,    2,    1,    0,    0],
       [   0, 1132,    3,    0,    0,    0,    0,    0,    0,    0],
       [   8,    0, 1011,    1,    0,    0,    2,    9,    1,    0],
       [   1,    0,    1,  988,    0,    7,    0,    5,    6,    2],
       [   1,    1,    0,    0,  951,    0,    5,    0,    1,   23],
       [   3,    0,    0,   10,    1,  861,    6,    2,    6,    3],
       [   3,    2,    0,    0,    0,    2,  951,    0,    0,    0],
       [   2,    8,    4,    0,    2,    0,    0, 1002,    0,   10],
       [   3,    0,    2,    6,    1,    4,    5,    4,  945,    4],
       [   6,    6,    1,    3,    4,    3,    0,    4,    2,  980]],
      dtype=int64)

In [49]:
X = change_im(X_train, 'rotate', 10)
y = np.copy(y_train)
for i in range(2):
    y = np.concatenate((y, y_train))
print(1)

m = KNNClassifier(4, 'brute', 'cosine', True, 256)
m.fit(X, y)
print(2)
res = m.predict(X_test)

1
2


In [ ]:
np.mean(res == y_test)

In [51]:
res_rot_10= np.copy(res)
mist_rot_10 = metrics.confusion_matrix(y_test, res)
mist_rot_10

array([[ 976,    1,    0,    0,    0,    0,    2,    1,    0,    0],
       [   0, 1132,    2,    0,    0,    0,    0,    1,    0,    0],
       [   6,    2, 1008,    2,    1,    0,    1,   10,    2,    0],
       [   0,    0,    1,  986,    1,    7,    0,    3,    7,    5],
       [   1,    0,    0,    0,  952,    0,    5,    2,    0,   22],
       [   3,    0,    0,    6,    1,  874,    3,    2,    1,    2],
       [   2,    3,    0,    0,    0,    1,  951,    0,    1,    0],
       [   2,    7,    3,    0,    0,    0,    0, 1008,    0,    8],
       [   2,    0,    3,    5,    4,    3,    2,    3,  947,    5],
       [   4,    4,    1,    5,    5,    2,    1,    5,    3,  979]],
      dtype=int64)

In [61]:
X = change_im(X_train, 'gauss', 1)
y = np.copy(y_train)
for i in range(1):
    y = np.concatenate((y, y_train))
print(1)

m = KNNClassifier(4, 'brute', 'cosine', True, 256)
m.fit(X, y)
print(2)
res = m.predict(X_test)

1
2


In [62]:
np.mean(res == y_test)

0.9813

In [63]:
res_gauss1 = np.copy(res)
mist_gauss1 = metrics.confusion_matrix(y_test, res)
mist_gauss1

array([[ 977,    1,    0,    0,    0,    0,    0,    2,    0,    0],
       [   0, 1130,    3,    0,    0,    0,    2,    0,    0,    0],
       [   7,    1, 1006,    2,    1,    0,    1,   12,    2,    0],
       [   0,    0,    1,  986,    1,    9,    0,    4,    5,    4],
       [   0,    0,    0,    0,  960,    0,    4,    3,    0,   15],
       [   2,    1,    0,    6,    1,  872,    4,    1,    2,    3],
       [   3,    3,    0,    0,    1,    3,  948,    0,    0,    0],
       [   0,    7,    6,    0,    3,    1,    0, 1003,    0,    8],
       [   3,    0,    1,    3,    3,    3,    3,    4,  951,    3],
       [   2,    4,    0,    2,    8,    3,    1,    7,    2,  980]],
      dtype=int64)

In [11]:

def change_im_rand_best(X_train):
    rot = [-15, -10,-5, 5, 10, 15]
    shift = [-3, -2, -1, 1, 2, 3]
    gauss = [1, 1.5, 0.5]
    X = np.copy(X_train)
    
    a = random.randint(1, 3)
    res = np.empty((X.shape[0] * 4, X.shape[1]))
    res[0:X.shape[0]] = np.copy(X_train)
    pos = X.shape[0]
    
    for i in range(3):
        for im in X:
            if a == 1: #gauss
                b = random.randint(0, 1)
                res[pos] = filters.gaussian(im.reshape(-1, 28), gauss[b]).reshape(1, -1)
                pos += 1

            if a == 2: #rotate
                b = random.randint(0, 3)
                res[pos] = transform.rotate(im.reshape(-1, 28), rot[b]).reshape(1, -1)
                pos += 1     

            if a == 3: #shift
                b = random.randint(0, 5)
                c = random.randint(0, 5)
                d = random.randint(0, 1)
                if d == 0:
                    b = 0
                else:
                    c = 0
                tform = SimilarityTransform(translation=(shift[b], shift[c]))
                res[pos] = transform.warp(im.reshape(-1, 28), tform).reshape(1, -1)
                pos += 1  
           
    return res
            
        

In [66]:
X = change_im_rand_best(X_train)
y = np.copy(y_train)
for i in range(3):
    y = np.concatenate((y, y_train))
print(1)

m = KNNClassifier(4, 'brute', 'cosine', True, 256)
m.fit(X, y)
print(2)
res = m.predict(X_test)

1
2


In [67]:
np.mean(res == y_test)

0.9787

In [68]:
res_rand = np.copy(res)
mist_rand = metrics.confusion_matrix(y_test, res)
mist_rand

array([[ 975,    1,    0,    0,    0,    1,    1,    1,    1,    0],
       [   0, 1131,    2,    0,    0,    1,    1,    0,    0,    0],
       [   6,    2, 1006,    3,    1,    0,    1,   11,    2,    0],
       [   0,    0,    1,  983,    1,   10,    0,    5,    5,    5],
       [   0,    0,    0,    0,  957,    0,    4,    3,    1,   17],
       [   1,    2,    0,    9,    1,  868,    5,    1,    2,    3],
       [   3,    3,    0,    0,    1,    3,  947,    0,    1,    0],
       [   0,    7,    4,    2,    4,    0,    1, 1000,    0,   10],
       [   2,    0,    5,    7,    3,    5,    5,    3,  940,    4],
       [   2,    4,    0,    1,   10,    4,    0,    6,    2,  980]],
      dtype=int64)

In [97]:

def change_im_all(X_train):
    rot = [-10, 10]
    shift = [-1,0, 1]
    gauss = [1]
    X = np.copy(X_train)
    
    res = np.empty((X.shape[0] * 8, X.shape[1]))
    res[0:X.shape[0]] = np.copy(X_train)
    pos = X.shape[0]
    
    for im in X:
        res[pos] = filters.gaussian(im.reshape(-1, 28), 1).reshape(1, -1)
        pos += 1
            
    for im in X:
        res[pos] = transform.rotate(im.reshape(-1, 28), -10).reshape(1, -1)
        pos += 1
    for im in X:
        res[pos] = transform.rotate(im.reshape(-1, 28), 10).reshape(1, -1)
        pos += 1

        
    for im in X:
        tform = SimilarityTransform(translation=(0, 1))
        res[pos] = transform.warp(im.reshape(-1, 28), tform).reshape(1, -1)
        pos += 1  
    for im in X:
        tform = SimilarityTransform(translation=(1, 0))
        res[pos] = transform.warp(im.reshape(-1, 28), tform).reshape(1, -1)
        pos += 1 
    for im in X:
        tform = SimilarityTransform(translation=(-1, 0))
        res[pos] = transform.warp(im.reshape(-1, 28), tform).reshape(1, -1)
        pos += 1             
    for im in X:
        tform = SimilarityTransform(translation=(0, -1))
        res[pos] = transform.warp(im.reshape(-1, 28), tform).reshape(1, -1)
        pos += 1 
           
    return res
            
        

In [98]:
X = change_im_all(X_train)
y = np.copy(y_train)
for i in range(7):
    y = np.concatenate((y, y_train))
print(1)

m = KNNClassifier(4, 'brute', 'cosine', True, 128)
m.fit(X, y)
print(2)
res = m.predict(X_test)

1
2


In [99]:
np.mean(res == y_test)

0.985

In [100]:
res_best = np.copy(res)
mist_best = metrics.confusion_matrix(y_test, res)
mist_best

array([[ 976,    0,    0,    0,    0,    0,    3,    1,    0,    0],
       [   0, 1132,    3,    0,    0,    0,    0,    0,    0,    0],
       [   5,    1, 1013,    0,    1,    0,    2,   10,    0,    0],
       [   0,    0,    2,  992,    1,    4,    0,    3,    4,    4],
       [   0,    0,    0,    0,  962,    0,    4,    2,    0,   14],
       [   2,    0,    0,    5,    1,  875,    6,    1,    0,    2],
       [   2,    2,    0,    0,    0,    1,  953,    0,    0,    0],
       [   1,    7,    4,    0,    0,    0,    0, 1009,    0,    7],
       [   2,    0,    3,    2,    2,    4,    3,    4,  952,    2],
       [   1,    3,    1,    3,    5,    2,    1,    4,    3,  986]],
      dtype=int64)

In [59]:
def print_table(a, n=10, m=10):
    print("\\begin{table}[h]")
    print("\\begin{center}")

    print("\\begin{{tabular}}{{|{0}|}}".format("l" * m))
    print("\\hline")
    for i in range(n):
        for j in range(m):
            if j > 0:
                print("& ", end="")
            print(a[i, j], end=" ")
        if i < n:
            print(" \\\\")
        else:
            print()
    print("\\hline")

    print("\\end{tabular}")
    print("\\end{center}")

    print("\\end{table}")
